# Quiz - Managing Conversation History
- trim_messages()
- RunnablePassthrough
- itemgetter

session_id 지정하여 대화 진행.
llm 모델이 과거 대화를 알지 못하는 상황 만들기.

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import SystemMessage, HumanMessage, trim_messages
from operator import itemgetter
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name, 'r', encoding='utf-8') as f:
    law = f.read()

llm = ChatOpenAI(
    model = 'gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    )

template = [
    ('system', '''당신은 육아휴직 법률 전문가입니다.
        아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
        답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
        - 문서: {law}'''),
        MessagesPlaceholder(variable_name="messages"),
        ('user', '{query}')
    ]

prompt = ChatPromptTemplate.from_messages(template)

trimmer = trim_messages(
    max_tokens=5_000,
    strategy="last",
    token_counter=llm,
    include_system=True, #Systemmessage 포함 출력
    allow_partial=False,
    start_on="Human",
)

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    |prompt
    |llm
)

# 세션별 대화 이력을 임시 메모리 저장
store = {}

config = {"configurable":{"session_id":"asdf"} }

def get_session_history(session_id: str) -> BaseChatMessageHistory: #부모클래스
	if session_id not in store:
		store[session_id] = InMemoryChatMessageHistory() #자식클래스
	return store[session_id]

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="query",
    history_messages_key="messages"
)

while True:
    query = input('질문을 입력하시오. 관두려면 S')
    if query.upper() == 'S':
        break
    response = with_message_history.invoke(
        {"query": query, "law": law},
        config=config,
    )
    print('\n' + '=' * 50 + '\n')  # 이 줄이 동일한 블록 안으로 들어가야 함


육아휴직의 기간은 1년 이내로 정해져 있습니다. 다만, 같은 자녀를 대상으로 부모가 모두 육아휴직을 각각 3개월 이상 사용한 경우나 한부모가족의 부 또는 모, 또는 장애아동의 부 또는 모인 경우에는 6개월 이내에서 추가로 육아휴직을 사용할 수 있습니다. (제19조 제2항)

육아휴직에 대한 조건은 제19조에 설명되어 있습니다. 사업주는 근로자가 임신 중인 여성 근로자이거나, 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위하여 육아휴직을 신청하는 경우 이를 허용해야 합니다. 다만, 대통령령으로 정하는 특정한 경우에는 예외가 있을 수 있습니다. 육아휴직의 기간은 1년 이내로 하며, 특정 조건에 따라 최대 6개월을 추가로 사용할 수 있습니다. 육아휴직을 이유로 한 해고 또는 불리한 처우는 금지되어 있고, 휴직 후에는 같은 수준의 임금을 지급하는 직무로 복귀시켜야 합니다(제19조).

육아휴직의 대상자는 임신 중인 여성 근로자 또는 만 8세 이하이거나 초등학교 2학년 이하의 자녀를 양육하는 근로자입니다. 이 법에 따라 사업주는 임신 중인 여성 근로자가 모성을 보호하거나 해당 조건을 충족하는 근로자가 육아휴직을 신청하는 경우 이를 허용해야 합니다. 단, 대통령령으로 정하는 경우에는 육아휴직이 허용되지 않을 수 있습니다. (남녀고용평등법 제19조)

육아휴직 수당에 대한 구체적인 금액이나 지급 기준은 "남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률"에서 구체적으로 명시되어 있지 않습니다. 그러나 이 법의 제19조에서는 육아휴직에 대한 규정을 두고 있습니다. 육아휴직 기간 중에는 해고나 불리한 처우가 금지되며, 휴직 후에는 같은 업무나 같은 수준의 임금이 지급되는 직무로 복귀해야 한다고 명시되어 있습니다.

육아휴직 수당에 관한 더 구체적인 정보는 "고용보험법" 등 다른 관련 법률이나 고용노동부의 지침에서 확인할 수 있습니다. 궁금하신 금액이나 기준을 확인하시려면 고용노동부나 해당 기관에 문의하시는 것이 좋겠습니다.

저는 사용자의 첫 번째 질문을 기억할 수

{'asdf': InMemoryChatMessageHistory(messages=[HumanMessage(content='육아휴직 기간은?', additional_kwargs={}, response_metadata={}), AIMessage(content='육아휴직의 기간은 1년 이내로 정해져 있습니다. 다만, 같은 자녀를 대상으로 부모가 모두 육아휴직을 각각 3개월 이상 사용한 경우나 한부모가족의 부 또는 모, 또는 장애아동의 부 또는 모인 경우에는 6개월 이내에서 추가로 육아휴직을 사용할 수 있습니다. (제19조 제2항)', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8'}, id='run-467295bd-8945-47d6-863a-5365e599e5b8-0'), HumanMessage(content='육아휴직 조건?', additional_kwargs={}, response_metadata={}), AIMessage(content='육아휴직에 대한 조건은 제19조에 설명되어 있습니다. 사업주는 근로자가 임신 중인 여성 근로자이거나, 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위하여 육아휴직을 신청하는 경우 이를 허용해야 합니다. 다만, 대통령령으로 정하는 특정한 경우에는 예외가 있을 수 있습니다. 육아휴직의 기간은 1년 이내로 하며, 특정 조건에 따라 최대 6개월을 추가로 사용할 수 있습니다. 육아휴직을 이유로 한 해고 또는 불리한 처우는 금지되어 있고, 휴직 후에는 같은 수준의 임금을 지급하는 직무로 복귀시켜야 합니다(제19조).', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerpr

#### 질문
- 육아휴직 기간?
- 조건?
- 대상자?
- 수당?
- 첫 번째 질문이 뭐야?
- 질문 정리해줘.
- 네 역할은?
- 육아휴직 질문에 대한 답변 정리.

In [6]:

# 이전 대화기록 전부 출력하기
for messages in get_session_history("asdf").messages:
	print(f'[{messages.type.upper()}]: {messages.content}')
	print()


[HUMAN]: 육아휴직 기간은?

[AI]: 육아휴직의 기간은 1년 이내로 정해져 있습니다. 다만, 같은 자녀를 대상으로 부모가 모두 육아휴직을 각각 3개월 이상 사용한 경우나 한부모가족의 부 또는 모, 또는 장애아동의 부 또는 모인 경우에는 6개월 이내에서 추가로 육아휴직을 사용할 수 있습니다. (제19조 제2항)

[HUMAN]: 육아휴직 조건?

[AI]: 육아휴직에 대한 조건은 제19조에 설명되어 있습니다. 사업주는 근로자가 임신 중인 여성 근로자이거나, 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위하여 육아휴직을 신청하는 경우 이를 허용해야 합니다. 다만, 대통령령으로 정하는 특정한 경우에는 예외가 있을 수 있습니다. 육아휴직의 기간은 1년 이내로 하며, 특정 조건에 따라 최대 6개월을 추가로 사용할 수 있습니다. 육아휴직을 이유로 한 해고 또는 불리한 처우는 금지되어 있고, 휴직 후에는 같은 수준의 임금을 지급하는 직무로 복귀시켜야 합니다(제19조).

[HUMAN]: 육아휴직 대상자?

[AI]: 육아휴직의 대상자는 임신 중인 여성 근로자 또는 만 8세 이하이거나 초등학교 2학년 이하의 자녀를 양육하는 근로자입니다. 이 법에 따라 사업주는 임신 중인 여성 근로자가 모성을 보호하거나 해당 조건을 충족하는 근로자가 육아휴직을 신청하는 경우 이를 허용해야 합니다. 단, 대통령령으로 정하는 경우에는 육아휴직이 허용되지 않을 수 있습니다. (남녀고용평등법 제19조)

[HUMAN]: 육아휴직 수당 어떻게 돼?

[AI]: 육아휴직 수당에 대한 구체적인 금액이나 지급 기준은 "남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률"에서 구체적으로 명시되어 있지 않습니다. 그러나 이 법의 제19조에서는 육아휴직에 대한 규정을 두고 있습니다. 육아휴직 기간 중에는 해고나 불리한 처우가 금지되며, 휴직 후에는 같은 업무나 같은 수준의 임금이 지급되는 직무로 복귀해야 한다고 명시되어 있습니다.

육아휴직 수당에 관한 더 구체적인 정보는 "고용보험법" 등 다

In [7]:
# trimmer 사용한 후 남은 대화 기록 
response.content

'귀하의 질문과 그것에 대한 답변을 정리한 내용은 다음과 같습니다.\n\n질문: 육아휴직 신청 및 사용과 관련하여 알아야 할 사항을 알려주세요.\n\n답변: 육아휴직에 대한 중요사항은 다음과 같습니다.\n\n1. **육아휴직 신청**: 사업주는 근로자가 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위해 육아휴직을 신청하는 경우 허용해야 합니다. 다만, 대통령령으로 정하는 특정한 경우에는 예외가 있습니다. (제19조 제1항)\n\n2. **육아휴직 기간**: 육아휴직은 1년 이내로 사용할 수 있습니다. 특정 조건을 충족하는 경우에는 추가로 6개월 이내에서 육아휴직을 더 사용할 수 있습니다. 해당 조건은 같은 자녀를 대상으로 부모가 모두 각각 3개월 이상 육아휴직을 사용한 경우, 한부모가족의 경우, 장애아동의 부모인 경우 등이 포함됩니다. (제19조 제2항)\n\n3. **육아휴직 후 복귀**: 사업주는 육아휴직을 마친 근로자를 휴직 전과 같은 업무 또는 같은 수준의 임금을 지급하는 직무에 복귀시켜야 하며, 육아휴직 기간은 근속 기간에 포함됩니다. (제19조 제4항)\n\n4. **보호 규정**: 사업주는 육아휴직을 이유로 근로자를 해고하거나 불리한 처우를 할 수 없습니다. 육아휴직 기간 중 해고도 금지되지만, 사업을 계속할 수 없는 경우는 예외입니다. (제19조 제3항)\n\n5. **신청 방법 및 절차**: 육아휴직의 신청방법 및 절차 등은 대통령령으로 정해집니다. (제19조 제6항)\n\n위의 내용은 남녀고용평등과 일·가정 양립 지원에 관한 법률 제19조에 해당하며, 육아휴직에 관한 구체적인 규정들을 설명하고 있습니다. 도움이 필요하시면 추가 질문 부탁드립니다.'